In [1]:
import os
os.environ["LANGGRAPH_DISABLE_DAG_RENDER"] = "1"
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from google import genai
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages

In [2]:
load_dotenv()

True

In [3]:
import os
api_key = os.getenv("GEMINI_API_KEY2") # or os.getenv("GEMINI_API_KEY")
if not api_key:
	raise EnvironmentError("GEMINI_API_KEY2 (or GEMINI_API_KEY) not set in environment. Add it to your .env or export it.")

# client = genai.Client(api_key=api_key)

In [4]:
model = ChatOpenAI(
   model="gemini-2.5-flash",
   api_key=api_key,
   base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [5]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):
    topic= state['topic']
    prompt= f'write humorous joke on following topic: {topic}'
    response= model.invoke(prompt).content
    return {"joke": response}

In [7]:
def explain_joke(state: JokeState):
    joke= state['joke']
    prompt= f'Explain the following joke: \n {joke}'
    resp= model.invoke(prompt).content
    return {'explanation': resp}

In [8]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('get_explanation', explain_joke)

graph.add_edge(START, 'generate_joke')
graph.add_edge("generate_joke", 'get_explanation')
graph.add_edge("get_explanation", END)

checkpointer= InMemorySaver()

joke_and_explain= graph.compile(checkpointer=checkpointer)

In [9]:
config1= {'configurable': {"thread_id": "1"}}

In [13]:
initial_state= {"topic": "Politics"}
final_state= joke_and_explain.invoke(initial_state, config= config1)

In [14]:
print(final_state['joke'])

Here are a few humorous jokes about politics:

1.  "How do you know when a politician is lying?"
    "Their lips are moving."

2.  "What's the difference between a politician and a baby?"
    "Eventually, you can reason with the baby."

3.  A politician, a lawyer, and an honest man walk into a bar.
    The bartender looks up and says, "Hey, an honest man! What can I get you?"

4.  "Why do politicians always talk about the future?"
    "Because it's easier to promise things that way."

5.  Politics is like a treadmill. You exert a lot of energy, feel like you're going somewhere, but ultimately end up in the same place.


In [15]:
print(final_state['explanation'])

These jokes all play on common stereotypes and cynical perceptions about politicians and the political process. They derive their humor from exaggeration, unexpected comparisons, and biting social commentary.

Let's break them down:

1.  **"How do you know when a politician is lying?" "Their lips are moving."**
    *   **Explanation:** This is a classic piece of hyperbole. It humorously exaggerates the widespread perception that politicians are inherently dishonest and lie constantly. The joke implies that anything a politician says is likely a lie, simply by virtue of them speaking.

2.  **"What's the difference between a politician and a baby?" "Eventually, you can reason with the baby."**
    *   **Explanation:** This joke uses an unflattering comparison. It suggests that politicians, despite being adults, are unreasonable, stubborn, and impervious to logic, facts, or the concerns of the public. A baby, though initially irrational, will eventually develop the capacity for reason, im

In [16]:
joke_and_explain.get_state(config1)

StateSnapshot(values={'topic': 'Politics', 'joke': 'Here are a few humorous jokes about politics:\n\n1.  "How do you know when a politician is lying?"\n    "Their lips are moving."\n\n2.  "What\'s the difference between a politician and a baby?"\n    "Eventually, you can reason with the baby."\n\n3.  A politician, a lawyer, and an honest man walk into a bar.\n    The bartender looks up and says, "Hey, an honest man! What can I get you?"\n\n4.  "Why do politicians always talk about the future?"\n    "Because it\'s easier to promise things that way."\n\n5.  Politics is like a treadmill. You exert a lot of energy, feel like you\'re going somewhere, but ultimately end up in the same place.', 'explanation': 'These jokes all play on common stereotypes and cynical perceptions about politicians and the political process. They derive their humor from exaggeration, unexpected comparisons, and biting social commentary.\n\nLet\'s break them down:\n\n1.  **"How do you know when a politician is lyin

In [18]:
list(joke_and_explain.get_state_history(config1))

[StateSnapshot(values={'topic': 'Politics', 'joke': 'Here are a few humorous jokes about politics:\n\n1.  "How do you know when a politician is lying?"\n    "Their lips are moving."\n\n2.  "What\'s the difference between a politician and a baby?"\n    "Eventually, you can reason with the baby."\n\n3.  A politician, a lawyer, and an honest man walk into a bar.\n    The bartender looks up and says, "Hey, an honest man! What can I get you?"\n\n4.  "Why do politicians always talk about the future?"\n    "Because it\'s easier to promise things that way."\n\n5.  Politics is like a treadmill. You exert a lot of energy, feel like you\'re going somewhere, but ultimately end up in the same place.', 'explanation': 'These jokes all play on common stereotypes and cynical perceptions about politicians and the political process. They derive their humor from exaggeration, unexpected comparisons, and biting social commentary.\n\nLet\'s break them down:\n\n1.  **"How do you know when a politician is lyi

In [10]:
config2= {'configurable': {"thread_id": "2"}}
initial_state= {"topic": "Bollywood"}
final_state2= joke_and_explain.invoke(initial_state, config= config2)

In [11]:
list(joke_and_explain.get_state_history(config2))

[StateSnapshot(values={'topic': 'Bollywood', 'joke': 'Why did the Bollywood actor bring a map of Switzerland to a family dinner?\n\nJust in case a dramatic revelation led to a spontaneous song and dance sequence!', 'explanation': 'This joke plays on several classic Bollywood tropes:\n\n1.  **Dramatic Revelations at Family Events:** Bollywood films are famous for their high drama, often involving long-lost relatives, secret love affairs, hidden parentage, or other major family secrets that come to light, usually with shouting, tears, and grand gestures. A family dinner is a perfect setting for such a "dramatic revelation."\n\n2.  **Spontaneous Song and Dance Sequences:** This is the absolute hallmark of Bollywood cinema. Characters frequently break into elaborate song and dance numbers, seemingly out of nowhere, to express intense emotions, advance the plot, or simply provide a colorful interlude. These sequences are often triggered by a significant emotional event – like a "dramatic re

### Time Traveling

In [12]:
config2_timeTravel= {'configurable': {"thread_id": "2", "checkpoint_id":"1f0c537a-1157-6174-8001-b9d6a8373253"}}

In [13]:
joke_and_explain.invoke(None, {'configurable': {"thread_id": "2", "checkpoint_id":"1f0c537a-1157-6174-8001-b9d6a8373253"}})

{'topic': 'Bollywood',
 'joke': 'Why did the Bollywood actor bring a map of Switzerland to a family dinner?\n\nJust in case a dramatic revelation led to a spontaneous song and dance sequence!',
 'explanation': "This joke plays on several common stereotypes and tropes associated with **Bollywood films**:\n\n1.  **Dramatic Revelations:** Bollywood plots are famous for their often over-the-top, emotional, and dramatic twists and turns, especially within family dynamics (long-lost relatives, secret loves, betrayals, etc.).\n2.  **Spontaneous Song and Dance Sequences:** A hallmark of Bollywood is that characters frequently burst into elaborate song and dance numbers to express their emotions, move the plot forward, or simply for entertainment. These sequences often happen in picturesque, exotic locations.\n3.  **Switzerland as a Filming Location:** Switzerland, with its beautiful mountains, lakes, and scenic landscapes, has been a hugely popular and iconic backdrop for countless Bollywood s

In [ ]:
# Old explanation
print(final_state2['explanation'])

This joke plays on several classic Bollywood tropes:

1.  **Dramatic Revelations at Family Events:** Bollywood films are famous for their high drama, often involving long-lost relatives, secret love affairs, hidden parentage, or other major family secrets that come to light, usually with shouting, tears, and grand gestures. A family dinner is a perfect setting for such a "dramatic revelation."

2.  **Spontaneous Song and Dance Sequences:** This is the absolute hallmark of Bollywood cinema. Characters frequently break into elaborate song and dance numbers, seemingly out of nowhere, to express intense emotions, advance the plot, or simply provide a colorful interlude. These sequences are often triggered by a significant emotional event – like a "dramatic revelation."

3.  **Exotic Filming Locations for Song Sequences:** Many Bollywood song sequences are shot in breathtaking, picturesque international locations, even if the rest of the film is set in India. **Switzerland** is one of the *

In [17]:
## kept same joke but explanation generated newly(above)
print(final_state2['joke'])

Why did the Bollywood actor bring a map of Switzerland to a family dinner?

Just in case a dramatic revelation led to a spontaneous song and dance sequence!


###  Update state
```
We can update state values
```

In [22]:
updated_final_state = joke_and_explain.update_state({"configurable": {'thread_id':"2", 'checkpoint_id': '1f0c5379-a49f-6b12-8000-54b788793a5a', 'checkpoint_ns': ""}}, {'topic': 'Data Science'})

In [23]:
updated_final_state

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0c5398-8a7d-6b4f-8001-9075c9b0ba9f'}}

In [24]:
list(joke_and_explain.get_state_history(config2))

[StateSnapshot(values={'topic': 'Data Science'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5398-8a7d-6b4f-8001-9075c9b0ba9f'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-11-19T11:18:54.499515+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5379-a49f-6b12-8000-54b788793a5a'}}, tasks=(PregelTask(id='387e53cc-688e-d532-a748-1de7fed7a1fd', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'Bollywood', 'joke': 'Why did the Bollywood actor bring a map of Switzerland to a family dinner?\n\nJust in case a dramatic revelation led to a spontaneous song and dance sequence!', 'explanation': "This joke plays on several common stereotypes and tropes associated with **Bollywood films**:\n\n1.  **Dramatic Revelations:** Bollywood plots 

In [28]:
updated_final_state_gen = joke_and_explain.invoke(None, {"configurable": {'thread_id':"2", 'checkpoint_id': '1f0c5398-8a7d-6b4f-8001-9075c9b0ba9f'}})

In [29]:
list(joke_and_explain.get_state_history(config2))

[StateSnapshot(values={'topic': 'Data Science', 'joke': 'Here are a few humorous jokes about data science:\n\n1.  Why did the data scientist break up with their dataset?\n    **It had too many missing values and was constantly trying to normalize their relationship!**\n\n2.  What do you call a data scientist who believes correlation implies causation?\n    **Naive Bayes!**\n\n3.  A data scientist, a statistician, and a business analyst walk into a bar.\n    The business analyst says, "We need actionable insights!"\n    The statistician says, "We need to account for confounding variables!"\n    The data scientist sighs and says, **"I just need to figure out which column is the actual timestamp."**\n\n4.  Why did the data scientist get kicked out of the garden party?\n    **They kept trying to prune the rose bushes for overfitting.**\n\n5.  Manager asks the data scientist: "Can you predict next quarter\'s sales?"\n    Data Scientist: "Yes! With 99% accuracy... of what happened last quart

In [ ]:
# next step --> backend- langgraph and frontend- Streamlit